In [4]:
from __future__ import division
import numpy as np
import scipy.linalg as slg
import math
import sys
from numpy import genfromtxt
import csv
from numpy import linalg as LA
from numpy.linalg import inv

## Output Processing Functions

In [5]:
def str2float(s):
    return float(s.replace('D','e'))

def CutLTMatOut(fname, head_line, index=0):
    cur_index = 0
    f = open(fname)
    status = None
    r = 0
    indices = []
    mat = []
    for line in f:
        if head_line in line:
            status = "Column Indices"
            continue
        if status == "Rows":
            splitted = line.split()
            if all([x.isnumeric() for x in splitted]):
                status = "Column Indices"
            else:
                try:
                    r = int(splitted[0])
                    floats = [str2float(s) for s in splitted[1:]]
                    if len(mat) < r:
                        mat.append(floats)
                    else:
                        mat[r-1] += floats
                except ValueError as e:
                    break
                continue
        if status == "Column Indices":
            status = "Rows"
            indices = [int(I) for I in line.split()]
            continue
    f.close()
    return mat

def toNPMatrix(mat_array, symmetry = 'symmetric'):
    dim = len(mat_array)
    mat = np.zeros((dim,dim))
    for i in range(dim):
        mat[i,:i+1] = mat_array[i]
    if symmetry == 'symmetric':
        diag = np.diag(np.diag(mat))
        return mat + mat.T - diag
    elif symmetry == 'anti-symmetric':
        return mat - mat.T


def ReadMatrix(fname, head_line, index=0):
    return np.array(CutLTMatOut(fname, head_line, index))

def ReadLowerTri(fname, head_line, index=0, symmetry = 'symmetric'):
    return toNPMatrix(CutLTMatOut(fname, head_line, index), symmetry)

def outMatrix(A):
    if np.iscomplexobj(A):
        print("Real part:")
        outMatrix(A.real)
        print()
        print("Imag part:")
        outMatrix(A.imag)
        return
    print("--------------------------------------------------------------------------------")
    curColStart, curColEnd = 0, 5
    while curColStart < A.shape[1]:
        curColEnd = min(curColStart + 5, A.shape[1])
        print("     "+"               ".join(list(map(str, range(curColStart + 1, curColEnd + 1)))))
        for i in range(A.shape[0]):
            print("%3d   "%(i+1,) + " ".join(list(map(lambda x : "%15.8e"%(x if abs(x) > 1e-8 else 0,) , A[i, curColStart : curColEnd]))))
        curColStart += 5
        print()

    print("--------------------------------------------------------------------------------")

def RMSD(V, W):
    """
    Calculate Root-mean-square deviation from two sets of vectors V and W.
    Parameters
    ----------
    V : array
        (N,D) matrix, where N is points and D is dimension.
    W : array
        (N,D) matrix, where N is points and D is dimension.
    Returns
    -------
    rmsd : float
        Root-mean-square deviation between the two vectors
    """
    diff = np.array(V) - np.array(W)
    N = len(V)
    return np.sqrt((diff * diff).sum() / N)
    

# Error Matrix Calculation Functions

In [15]:
def getErrorMatrix(F,P,S,V,printlevel):
    FPS = np.dot(np.dot(F, P), S)

    e_AO = FPS - FPS.transpose()

    e_ON = np.dot(V.transpose(), np.dot(e_AO, V))

    if (printlevel >= 1):
        if (printlevel >= 2):
            print('F')
            outMatrix(F)
            print('P')
            outMatrix(P)
            print('FP')
            outMatrix(np.dot(F, P))
            print('S')   
            outMatrix(S)
        print('FPS')
        outMatrix(FPS)
        print('e_AO')
        outMatrix(e_AO)
        if (printlevel >= 2):
            print('V')
            outMatrix(V)
        print('e_ON')
        outMatrix(e_ON)
    return  e_ON

In [16]:
def getBmatrix(E_list):
    Bsize = len(E_list)
    B = np.zeros((Bsize+1,Bsize+1))
    
    B[0,0] = 0.00
    for i in range(1,Bsize+1):
        B[i,0] = -1
        B[0,i] = -1
        B[i,i] = np.trace(np.dot(E_list[i-1].T,E_list[i-1]))
        for j in range(i+1,Bsize+1):
            B[i,j] = np.trace(np.dot(E_list[i-1].T,E_list[j-1]))
            B[j,i] = B[i,j]
        
    return B

In [17]:
def newF(F_list, B):
    Fsize = len(F_list)
    NBasis = F_list[0].shape[0]
    
    F = np.zeros((NBasis,NBasis))
    for i in range(Fsize):
        ci = inv(B)[:,0][1:][i] * (-1)
        print('c' + str(i) + ': ', ci)
        F += ci * F_list[i]
    return F

## Manually Calculating B-matrix by Reading QC's F and P

In [18]:
fname_fulllog = "/Users/aodongliu/LiGroup/gaussian/manual_calculation/neo_small_basis_stw.log"
fname = "./Formaldehyde_STWDIIS.out"
prot_S_headline = 'Prot S Matrix:          0:'
prot_V_headline = 'V for IR=Prot          1'
prot_S = ReadLowerTri(fname_fulllog, prot_S_headline)
prot_V = ReadMatrix(fname_fulllog, prot_V_headline)


In [19]:
Fp1_headline = 'Fock Matrix1:'
Fp1 = ReadMatrix(fname, Fp1_headline)
Pp1_headline = 'Density Matrix1:'
Pp1 = ReadMatrix(fname, Pp1_headline)

e1_manual_ON = getErrorMatrix(Fp1,Pp1,prot_S,prot_V,0)
E_list = [e1_manual_ON]
F_list = [Fp1]

B = getBmatrix(E_list)
print('Manually Calculated B-matrix')
outMatrix(B)

newF_B1 = newF(F_list,B)

Manually Calculated B-matrix
--------------------------------------------------------------------------------
     1               2
  1    0.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00  5.74531055e-03

--------------------------------------------------------------------------------
c0:  1.0


In [20]:
Fp2_headline = 'Fock Matrix2:'
Fp2 = ReadMatrix(fname, Fp2_headline)
Pp2_headline = 'Density Matrix2:'
Pp2 = ReadMatrix(fname, Pp2_headline)

e2_manual_ON = getErrorMatrix(Fp2,Pp2,prot_S,prot_V,0)
E_list.append(e2_manual_ON)
F_list.append(Fp2)

B = getBmatrix(E_list)
print('Manually Calculated B-matrix')
outMatrix(B)

newF_B2 = newF(F_list,B)

Manually Calculated B-matrix
--------------------------------------------------------------------------------
     1               2               3
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00  5.74531055e-03  5.39736505e-03
  3   -1.00000000e+00  5.39736505e-03  5.07161404e-03

--------------------------------------------------------------------------------
c0:  -14.677109448504057
c1:  15.677109448504057


In [21]:
Fp3_headline = 'Fock Matrix3:'
Fp3 = ReadMatrix(fname, Fp3_headline)
Pp3_headline = 'Density Matrix3:'
Pp3 = ReadMatrix(fname, Pp3_headline)

e3_manual_ON = getErrorMatrix(Fp3,Pp3,prot_S,prot_V,0)
E_list.append(e3_manual_ON)
F_list.append(Fp3)

B = getBmatrix(E_list)
print('Manually Calculated B-matrix')
outMatrix(B)

newF_B3 = newF(F_list,B)

Manually Calculated B-matrix
--------------------------------------------------------------------------------
     1               2               3               4
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00  5.74531055e-03  5.39736505e-03  3.15496953e-04
  3   -1.00000000e+00  5.39736505e-03  5.07161404e-03  3.11951568e-04
  4   -1.00000000e+00  3.15496953e-04  3.11951568e-04  2.40086021e-04

--------------------------------------------------------------------------------
c0:  28.814033363178726
c1:  -30.80983421702675
c2:  2.995800853848024


In [22]:
Fp4_headline = 'Fock Matrix4:'
Fp4 = ReadMatrix(fname, Fp4_headline)
Pp4_headline = 'Density Matrix4:'
Pp4 = ReadMatrix(fname, Pp4_headline)

e4_manual_ON = getErrorMatrix(Fp4,Pp4,prot_S,prot_V,0)
E_list.append(e4_manual_ON)
F_list.append(Fp4)

B = getBmatrix(E_list)
print('Manually Calculated B-matrix')
outMatrix(B)

newF_B4 = newF(F_list,B)

Manually Calculated B-matrix
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00  5.74531055e-03  5.39736505e-03  3.15496953e-04  1.22133714e-04
  3   -1.00000000e+00  5.39736505e-03  5.07161404e-03  3.11951568e-04  1.28420628e-04
  4   -1.00000000e+00  3.15496953e-04  3.11951568e-04  2.40086021e-04  2.02859288e-04
  5   -1.00000000e+00  1.22133714e-04  1.28420628e-04  2.02859288e-04  1.75329246e-04

--------------------------------------------------------------------------------
c0:  -3.378764171173729
c1:  3.813069512997166
c2:  -6.400229300873277
c3:  6.96592395904984


In [23]:
Fp5_headline = 'Fock Matrix5:'
Fp5 = ReadMatrix(fname, Fp5_headline)
Pp5_headline = 'Density Matrix5:'
Pp5 = ReadMatrix(fname, Pp5_headline)

e5_manual_ON = getErrorMatrix(Fp5,Pp5,prot_S,prot_V,0)
E_list.append(e5_manual_ON)
F_list.append(Fp5)

B = getBmatrix(E_list)
print('Manually Calculated B-matrix')
outMatrix(B)

newF_B5 = newF(F_list,B)

Manually Calculated B-matrix
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00  5.74531055e-03  5.39736505e-03  3.15496953e-04  1.22133714e-04
  3   -1.00000000e+00  5.39736505e-03  5.07161404e-03  3.11951568e-04  1.28420628e-04
  4   -1.00000000e+00  3.15496953e-04  3.11951568e-04  2.40086021e-04  2.02859288e-04
  5   -1.00000000e+00  1.22133714e-04  1.28420628e-04  2.02859288e-04  1.75329246e-04
  6   -1.00000000e+00  9.05828884e-07  8.23394182e-07 -3.48498690e-07 -3.31549499e-07

     6
  1   -1.00000000e+00
  2    9.05828884e-07
  3    8.23394182e-07
  4   -3.48498690e-07
  5   -3.31549499e-07
  6    0.00000000e+00

--------------------------------------------------------------------------------
c0:  0.03605603398981608
c1:  -0.04059160380440345
c2:  0.05870702837942072
c3:  

## Manually Form Protonic Density Matrix for QC

In [102]:
def compute_density(C, nbasis, NProt):
    P = np.zeros((nbasis,nbasis)) 
    for sig in range(nbasis):
        for lam in range(nbasis): 
            P[sig, lam] = 0.00
            for i in range(0, NProt):  
                P[sig, lam] += C[lam, i]*C[sig,i]
    return P*2

In [103]:
def compute_MO(F, V, nbasis):
    F_prime =  np.dot(np.dot(V.T, F), V)

    epsilon, C_prime  = LA.eigh(F_prime)
    
    C = np.dot(V, C_prime)
    return C

In [116]:
print("Q-Chem Fock matrix at Iteration 1")
outMatrix(Fp1)

Q-Chem Fock matrix at Iteration 1
--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -9.18184100e-01 -1.14653900e-01  0.00000000e+00  3.34838700e-01  0.00000000e+00
  2   -1.14653900e-01  5.88567300e-01  0.00000000e+00  1.92540000e-03  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00  5.64247600e-01  0.00000000e+00  0.00000000e+00
  4    3.34838700e-01  1.92540000e-03  0.00000000e+00  5.03341800e-01  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -9.18184100e-01
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.14653900e-01
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  3.34838700e-01

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.0000000

In [117]:
nbasis = 8
NProt = 2 
C = compute_MO(Fp1, prot_V, nbasis)

print("Coefficient Obtained by Diagonalizing Q-Chem Fock matrix at Iteration 1")
outMatrix(C)

Coefficient Obtained by Diagonalizing Q-Chem Fock matrix at Iteration 1
--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -9.73325980e-01  2.57620019e-02  1.31241899e-01  5.15713708e-02  3.95443566e-08
  2   -7.04656811e-02  1.86519755e-03  5.22171119e-01  2.05187496e-01  1.68468673e-07
  3    0.00000000e+00  1.32021665e-08  1.96263025e-05 -4.97741760e-05  6.43441972e-01
  4    2.16728348e-01 -5.73636460e-03  7.59181567e-01  2.98319751e-01  2.32832635e-07
  5   -2.57620012e-02 -9.73325975e-01 -5.15714260e-02  1.31241870e-01  5.22575380e-06
  6    1.86519738e-03  7.04656811e-02  2.05187307e-01 -5.22171445e-01 -1.78395170e-05
  7    0.00000000e+00  0.00000000e+00  3.49433763e-07  2.88045053e-07 -7.65495403e-01
  8    5.73637068e-03  2.16728264e-01 -2.98319856e-01  7.59181385e-01  2.92912793e-05

     6               7               8
  1   -4.46130538e-08  1.50938246e-01 -9.6470791

In [118]:
P2Manual = compute_density(C, nbasis, NProt)
outMatrix(P2Manual)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    1.89605429e+00  1.37268259e-01  0.00000000e+00 -4.22190224e-01  0.00000000e+00
  2    1.37268259e-01  9.93778235e-03  0.00000000e+00 -3.05652202e-02 -2.16522630e-07
  3    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.57189395e-08
  4   -4.22190224e-01 -3.05652202e-02  0.00000000e+00  9.40081655e-02  0.00000000e+00
  5    0.00000000e+00 -2.16522630e-07 -2.57189395e-08  0.00000000e+00  1.89605427e+00
  6   -2.16106774e-07  0.00000000e+00  0.00000000e+00  4.86159672e-08 -1.37268258e-01
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  4.75200816e-08  0.00000000e+00  0.00000000e+00 -4.22190058e-01

     6               7               8
  1   -2.16106774e-07  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000000e+00  4.75200816e-08
  3    0.000

In [107]:
print(RMSD(P2Manual,Pp2))

2.9099362966464084e-07


In [119]:
C_B2 = compute_MO(newF_B2, prot_V, nbasis)
P3Manual = compute_density(C_B2, nbasis, NProt)
print(RMSD(P3Manual,Pp3))

1.2733991322597623e-05


In [120]:
C_B3 = compute_MO(newF_B3, prot_V, nbasis)
P4Manual = compute_density(C_B3, nbasis, NProt)
print(RMSD(P4Manual,Pp4))

3.7046745869741455e-05


In [121]:
C_B4 = compute_MO(newF_B4, prot_V, nbasis)
P5Manual = compute_density(C_B4, nbasis, NProt)
print(RMSD(P5Manual,Pp5))

1.4262176422946815e-05


## Manually Form Protonic Density Matrix for Gaussian

In [111]:
tempF =  ReadLowerTri('temp.txt', 'NewF from SCFDIS:          0:') 

In [112]:
outMatrix(tempF)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -7.61727000e+00  3.60012000e-01  0.00000000e+00 -2.71172000e-01  0.00000000e+00
  2    3.60012000e-01 -7.50947000e+00  0.00000000e+00  3.41655000e-02  0.00000000e+00
  3    0.00000000e+00  0.00000000e+00 -7.45475000e+00  0.00000000e+00  0.00000000e+00
  4   -2.71172000e-01  3.41655000e-02  0.00000000e+00 -7.48927000e+00  0.00000000e+00
  5    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -7.61727000e+00
  6    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -3.60012000e-01
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.71172000e-01

     6               7               8
  1    0.00000000e+00  0.00000000e+00  0.00000000e+00
  2    0.00000000e+00  0.00000000e+00  0.00000000e+00
  3    0.000

In [113]:
nbasis = 8
NProt = 2 
C = compute_MO(tempF, prot_V, nbasis)
outMatrix(C)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1   -5.65762419e-02 -7.36372241e-01  3.65594800e-04 -5.53665278e-04  1.25614414e-08
  2    4.12718314e-02  5.37149940e-01  3.31618288e-01 -5.02223977e-01  4.81075757e-07
  3    2.70331898e-06 -2.06057874e-07  1.24493951e-04  8.23845386e-05  7.65337420e-01
  4   -3.10535222e-02 -4.04175277e-01  4.40052261e-01 -6.66448785e-01  6.32989088e-07
  5   -7.36372310e-01  5.65763166e-02  5.53686413e-04  3.66294822e-04  3.07178437e-06
  6   -5.37149246e-01  4.12719087e-02 -5.02225220e-01 -3.31619259e-01  6.80794130e-05
  7    0.00000000e+00  0.00000000e+00 -8.64734213e-07  6.95036745e-07  6.43629922e-01
  8   -4.04176169e-01  3.10532756e-02  6.66447862e-01  4.40051505e-01 -9.13166032e-05

     6               7               8
  1    1.24957149e-08  6.72464139e-01  4.84432525e-02
  2    4.31587686e-07  5.88025420e-01  4.23580533e-02
  3   -6.436

In [114]:
P1Manual = compute_density(C, nbasis, NProt)
outMatrix(P1Manual/2)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    5.45444948e-01 -3.97877310e-01  0.00000000e+00  2.99380346e-01 -5.10791606e-08
  2   -3.97877310e-01  2.90233422e-01  0.00000000e+00 -2.18384361e-01 -1.46873640e-06
  3    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.00230724e-06
  4    2.99380346e-01 -2.18384361e-01  0.00000000e+00  1.64321976e-01  2.05435404e-07
  5   -5.10791606e-08 -1.46873640e-06 -2.00230724e-06  2.05435404e-07  5.45445059e-01
  6   -1.60223742e-06  7.02050156e-08 -1.46059015e-06 -7.09119307e-07  3.97876844e-01
  7    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8    0.00000000e+00 -8.25578835e-07 -1.09901588e-06  1.27375667e-07  2.99381020e-01

     6               7               8
  1   -1.60223742e-06  0.00000000e+00  0.00000000e+00
  2    7.02050156e-08  0.00000000e+00 -8.25578835e-07
  3   -1.460

In [95]:
np.trace(P1Manual)

4.000001539350061

In [52]:
dim = 12
lenvec = 144
raw_vec = [2.07180697154,
-0.22194607781,
-0.00000000000,
-0.00000000000,
0.01799713641,
0.01191402458,
-0.00669405223,
0.00000000000,
0.00000000000,
0.14728047977,
-0.11088347368,
-0.11088347368,
-0.22194607781,
0.83595330929,
-0.00000000000,
0.00000000000,
-0.07620970563,
-0.00186046381,
-0.17558012068,
0.00000000000,
0.00000000000,
-0.41845256663,
0.29107087397,
0.29107087397,
-0.00000000000,
-0.00000000000,
0.63195977361,
0.00000000000,
0.00000000000,
0.00000000000,
-0.00000000000,
0.14056275723,
0.00000000000,
-0.00000000000,
0.44560156695,
-0.44560156695,
-0.00000000000,
0.00000000000,
0.00000000000,
0.75779457814,
0.00000000000,
0.00000000000,
-0.00000000000,
0.00000000000,
0.82578041826,
-0.00000000000,
-0.00000000000,
0.00000000000,
0.01799713641,
-0.07620970563,
0.00000000000,
0.00000000000,
0.55540197393,
-0.04249796935,
-0.00864361432,
-0.00000000000,
-0.00000000000,
-0.59352875102,
-0.25178176297,
-0.25178176297,
0.01191402458,
-0.00186046381,
0.00000000000,
0.00000000000,
-0.04249796935,
2.11065208892,
-0.46460455593,
0.00000000000,
0.00000000000,
-0.09556119710,
0.01025198722,
0.01025198722,
-0.00669405223,
-0.17558012068,
-0.00000000000,
-0.00000000000,
-0.00864361432,
-0.46460455593,
2.05091561975,
0.00000000000,
-0.00000000000,
0.55170134793,
-0.02562920309,
-0.02562920309,
0.00000000000,
0.00000000000,
0.14056275723,
0.00000000000,
-0.00000000000,
0.00000000000,
0.00000000000,
1.91972212018,
-0.00000000000,
-0.00000000000,
-0.35172130116,
0.35172130116,
0.00000000000,
0.00000000000,
0.00000000000,
0.82578041826,
-0.00000000000,
0.00000000000,
-0.00000000000,
-0.00000000000,
0.89986563490,
0.00000000000,
-0.00000000000,
0.00000000000,
0.14728047977,
-0.41845256663,
-0.00000000000,
-0.00000000000,
-0.59352875102,
-0.09556119710,
0.55170134793,
-0.00000000000,
0.00000000000,
1.03318837766,
0.11847151537,
0.11847151537,
-0.11088347368,
0.29107087397,
0.44560156695,
-0.00000000000,
-0.25178176297,
0.01025198722,
-0.02562920309,
-0.35172130116,
-0.00000000000,
0.11847151537,
0.61657149380,
-0.22708134196,
-0.11088347368,
0.29107087397,
-0.44560156695,
0.00000000000,
-0.25178176297,
0.01025198722,
-0.02562920309,
0.35172130116,
0.00000000000,
0.11847151537,
-0.22708134196,
0.61657149380]

In [61]:
mat = np.zeros((dim,dim))

for row in range(dim):
    for col in range(dim):
        mat[row,col] = raw_vec[col*dim+row]

In [62]:
outMatrix(mat)

--------------------------------------------------------------------------------
     1               2               3               4               5
  1    2.07180697e+00 -2.21946078e-01  0.00000000e+00  0.00000000e+00  1.79971364e-02
  2   -2.21946078e-01  8.35953309e-01  0.00000000e+00  0.00000000e+00 -7.62097056e-02
  3    0.00000000e+00  0.00000000e+00  6.31959774e-01  0.00000000e+00  0.00000000e+00
  4    0.00000000e+00  0.00000000e+00  0.00000000e+00  7.57794578e-01  0.00000000e+00
  5    1.79971364e-02 -7.62097056e-02  0.00000000e+00  0.00000000e+00  5.55401974e-01
  6    1.19140246e-02 -1.86046381e-03  0.00000000e+00  0.00000000e+00 -4.24979693e-02
  7   -6.69405223e-03 -1.75580121e-01  0.00000000e+00  0.00000000e+00 -8.64361432e-03
  8    0.00000000e+00  0.00000000e+00  1.40562757e-01  0.00000000e+00  0.00000000e+00
  9    0.00000000e+00  0.00000000e+00  0.00000000e+00  8.25780418e-01  0.00000000e+00
 10    1.47280480e-01 -4.18452567e-01  0.00000000e+00  0.00000000e+00 -5.9

In [67]:
count = 0
for i in range(dim):
    for j in range(i+1):
        print(mat[j,i])
        count+=1


2.07180697154
-0.22194607781
0.83595330929
-0.0
-0.0
0.63195977361
-0.0
0.0
0.0
0.75779457814
0.01799713641
-0.07620970563
0.0
0.0
0.55540197393
0.01191402458
-0.00186046381
0.0
0.0
-0.04249796935
2.11065208892
-0.00669405223
-0.17558012068
-0.0
-0.0
-0.00864361432
-0.46460455593
2.05091561975
0.0
0.0
0.14056275723
0.0
-0.0
0.0
0.0
1.91972212018
0.0
0.0
0.0
0.82578041826
-0.0
0.0
-0.0
-0.0
0.8998656349
0.14728047977
-0.41845256663
-0.0
-0.0
-0.59352875102
-0.0955611971
0.55170134793
-0.0
0.0
1.03318837766
-0.11088347368
0.29107087397
0.44560156695
-0.0
-0.25178176297
0.01025198722
-0.02562920309
-0.35172130116
-0.0
0.11847151537
0.6165714938
-0.11088347368
0.29107087397
-0.44560156695
0.0
-0.25178176297
0.01025198722
-0.02562920309
0.35172130116
0.0
0.11847151537
-0.22708134196
0.6165714938


In [68]:
count

78

In [69]:
np.trace(np.dot(Fp1,Pp1))

-4.003270108278